# VITK Project

Benjamin Clene
Yanis Farhat
Theo Bonzi

In [15]:
#!pip install -r requirements.txt
#!pip install vtk
#!pip install --user itkwidgets

In [140]:
import itk
import os
import matplotlib
import matplotlib.pyplot as plt
import sys
import argparse
import numpy as np
import vtk
from ipywidgets import interact, fixed
import ipywidgets as widgets
from IPython.display import display
from tqdm import tqdm

In [158]:
path1 = "Data/case6_gre1.nrrd"
path2 = "Data/case6_gre2.nrrd"
path3 = "Data/recalage.nrrd"
path4 = "Data/segmented1.nrrd"
path5 = "Data/segmented2.nrrd"

def display_nrrd(path):
    scan = itk.imread(path)
    def update_image(layer_index):
        plt.imshow(scan[layer_index,:, :], cmap='gray')
        plt.axis('off')
        plt.show()

    # Obtention de la région la plus grande possible de l'image
    region = scan.GetLargestPossibleRegion()
    size = region.GetSize()   
    print(size)
    # Création du slider
    layer_slider = widgets.IntSlider(min=0, max=size[2] - 1, value=10, description='Couche')

    # valeur du slider change
    widgets.interact(update_image, layer_index=layer_slider)
    
def from_list_to_nrrd(list_image, output):
    nrrd_image = itk.GetImageFromArray(list_image)
    itk.imwrite(nrrd_image,output)
    
#display_nrrd(path1)
#display_nrrd(path2)
display_nrrd(path3)
#display_nrrd(path4)
#display_nrrd(path5)

itkSize3 ([256, 256, 176])


interactive(children=(IntSlider(value=10, description='Couche', max=175), Output()), _dom_classes=('widget-int…

In [149]:
PixelType = itk.F
scan1 = itk.imread("Data/case6_gre1.nrrd",PixelType)
scan2 = itk.imread("Data/case6_gre2.nrrd",PixelType)

region1 = scan1.GetLargestPossibleRegion()
region2 = scan2.GetLargestPossibleRegion()

size_scan1 = region1.GetSize()
size_scan2 = region2.GetSize()
print(size_scan1)
print(size_scan2)

def recalage(fixed_image, moving_image):
    dimension = fixed_image.GetImageDimension()
    FixedImageType = itk.Image[itk.F,2]
    MovingImageType = itk.Image[itk.F,2]

    TransformType = itk.TranslationTransform[itk.D, dimension]
    initialTransform = TransformType.New()

    optimizer = itk.RegularStepGradientDescentOptimizerv4.New()

    optimizer.SetLearningRate(4)
    optimizer.SetMinimumStepLength(0.001)
    optimizer.SetNumberOfIterations(2)

    metric = itk.MeanSquaresImageToImageMetricv4[FixedImageType, MovingImageType].New()
    fixed_interpolation = itk.LinearInterpolateImageFunction[FixedImageType, itk.D].New()
    metric.SetFixedInterpolator(fixed_interpolation)

    registration = itk.ImageRegistrationMethodv4.New(FixedImage=fixed_image, MovingImage=moving_image, Metric=metric,
                                                     Optimizer=optimizer, InitialTransform=initialTransform)

    moving_initial_transform = TransformType.New()
    initial_parameters = moving_initial_transform.GetParameters()
    initial_parameters[0] = 0
    initial_parameters[1] = 0
    moving_initial_transform.SetParameters(initial_parameters)
    registration.SetMovingInitialTransform(moving_initial_transform)

    identity_transform = TransformType.New()
    identity_transform.SetIdentity()
    registration.SetFixedInitialTransform(identity_transform)

    registration.SetNumberOfLevels(1)

    registration.Update()

    transform = registration.GetTransform()
    final_parameters = transform.GetParameters()
    translation_along_x = final_parameters.GetElement(0)
    translation_along_y = final_parameters.GetElement(1)

    number_of_iterations = optimizer.GetCurrentIteration()

    best_value = optimizer.GetValue()

    CompositeTransformType = itk.CompositeTransform[itk.D, dimension]
    output_composite_transform = CompositeTransformType.New()
    output_composite_transform.AddTransform(moving_initial_transform)
    output_composite_transform.AddTransform(registration.GetModifiableTransform())

    resampler = itk.ResampleImageFilter.New(Input=moving_image, Transform=transform, UseReferenceImage=True,
                                            ReferenceImage=fixed_image)
    resampler.SetDefaultPixelValue(100)

    subtraction = itk.SubtractImageFilter(Input1=fixed_image, Input2=resampler)
    res = itk.GetArrayViewFromImage(subtraction)
    return res

itkSize3 ([256, 256, 176])
itkSize3 ([256, 256, 176])


In [150]:
# Appel de la fonction sur toutes les couches en X du scans
recalage_image = np.zeros((size_scan1[2],size_scan1[1],size_scan1[0]))
for index in tqdm(range(size_scan1[0])):
    image1 = scan1[:,index, :]
    image1_itk = itk.GetImageFromArray(image1)
    image2 = scan2[:,index,:]
    image2_itk = itk.GetImageFromArray(image2)

    recalage_image[:,index,:] = recalage(image1_itk, image2_itk)

print(recalage_image.shape)

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [00:02<00:00, 98.97it/s]

(176, 256, 256)


In [155]:
from_list_to_nrrd(recalage_image,"Data/recalage.nrrd")

In [161]:
def segmentation_threshold(path, output):
    # Load the image
    image = itk.imread(path)

    # Valeur Threshold
    lower_threshold = 125
    upper_threshold = 200

    threshold_filter = itk.BinaryThresholdImageFilter.New(image)
    threshold_filter.SetLowerThreshold(lower_threshold)
    threshold_filter.SetUpperThreshold(upper_threshold)
    threshold_filter.SetInsideValue(1)
    threshold_filter.SetOutsideValue(0)
    threshold_filter.Update()

    # Image Segmenter
    segmented_image = threshold_filter.GetOutput()
    itk.imwrite(segmented_image, output)
    
segmentation_threshold('new_Data/case6_gre2.nrrd','new_Data/segmented2.nrrd')